# Analisis de propeidades estaticas
- Energía cinética media -> Temperatura media
- Fuerza cuadratica media
- Presión
- ...

## Importación de librerías y lectura de datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os
os.getcwd()

'/private/var/mobile/Containers/Data/Application/8C70C489-A646-4256-A688-F892C67ED0B6/Documents/Repos/Simulacion-DInamica-Molecular/Notebooks'

In [3]:
from pathlib import Path
codigo ="110-160"

ruta_vx = Path("..") / "resultados" / "dinamica" / f"Simulacion_{codigo}/velocidad_x.txt"
ruta_vy = Path("..") / "resultados" / "dinamica" / f"Simulacion_{codigo}/velocidad_y.txt"
ruta_vz = Path("..") / "resultados" / "dinamica" / f"Simulacion_{codigo}/velocidad_z.txt"
ruta_gname2 = Path("..") / "resultados" / "dinamica" / f"Simulacion_{codigo}/gname2.txt"

ruta=ruta_gname2
print(ruta.exists())  # True o False
print(ruta.parent)    # Ruta al directorio padre
print(ruta.name)      # archivo.txt
print(ruta.stem)      # archivo

True
../resultados/dinamica/Simulacion_110-160
gname2.txt
gname2


In [4]:
pd.read_csv(ruta, sep=r'\s+')

,t,Ec,Ep,E_tot,W,T,P_corr,F2
0,0.01,1016.1460,-1681.177,-665.0314,-1008.6110,1.354861,0.307718,549.9444
1,0.02,1009.4510,-1674.483,-665.0322,-853.3452,1.345935,0.355010,573.0316
2,0.03,1002.6610,-1667.683,-665.0224,-705.5814,1.336881,0.399738,606.1855
3,0.04,997.5940,-1662.624,-665.0296,-592.0951,1.330125,0.434189,639.9411
4,0.05,996.7793,-1661.809,-665.0293,-546.2595,1.329039,0.448924,633.7288
...,...,...,...,...,...,...,...,...
4995,49.96,1034.5940,-1699.641,-665.0471,-1180.0290,1.379459,0.262877,562.2455
4996,49.97,1022.1360,-1687.186,-665.0498,-1002.7700,1.362848,0.313659,662.9936
4997,49.98,1014.4410,-1679.493,-665.0521,-903.5724,1.352587,0.341594,704.0887
4998,49.99,1014.1660,-1679.220,-665.0537,-923.7901,1.352222,0.334672,667.7893


In [5]:
codigos=["110-160","160-210","110-160","210-260","260-310","310-360","360-410","410-460","460-510","510-560"] #Tiempo de inicio de simulacion-Tiempo de fin de simulacion

DFS_vx=[]
DFS_vy=[]
DFS_vz=[]

DATOS=[]

for codigo in codigos:
    DFS_vx.append(pd.read_csv(ruta_vx, sep=r"\s+", header=None))
    DFS_vy.append(pd.read_csv(ruta_vy, sep=r"\s+", header=None))
    DFS_vz.append(pd.read_csv(ruta_vz, sep=r"\s+", header=None))
    DATOS.append(pd.read_csv(ruta_gname2,skiprows=1, sep=r"\s+", header=None, names=['t','Ec','Ep','E_tot','W','T','P_corr','F2']))

In [6]:
tiempos=[df.t.to_numpy() for df in DATOS]

tiempos[0]

array([1.000e-02, 2.000e-02, 3.000e-02, ..., 4.998e+01, 4.999e+01,
       5.000e+01])

### Formato de datos importados

- En las columnas tenemos las particulas de la caja etiquetadas del 0 al 499
- En las filas tenemos instantes sucesivos de tiempo grabados de nuestra simulacion.

In [8]:
DATOS[-1].iloc[:,1:].agg(["mean","std"])
DATOS[-1].max()

t           50.000000
Ec        1093.431000
Ep       -1629.125000
E_tot     -664.943600
W         -336.780100
T            1.457907
P_corr       0.523574
F2        1018.172000
dtype: float64

In [11]:
F2s_mean=[]
F2s_std=[]

Ecs_mean=[]
Ecs_std=[]

Ps_mean=[]
Ps_std=[]


for df in DATOS:
    F2s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","F2"])
    F2s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","F2"])

    Ecs_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","Ec"])
    Ecs_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","Ec"])

    Ps_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","P_corr"])
    Ps_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","P_corr"])


F2s_mean=np.array(F2s_mean)
F2s_std=np.array(F2s_mean)

Ecs_mean=np.array(Ecs_mean)
Ecs_std=np.array(Ecs_std)

Ps_mean=np.array(Ps_mean)
Ps_std=np.array(Ps_std)


"""
print("Ec=",)

print(F2s_mean)
print(F2s_std)
print(Ecs_mean)
print(Ecs_std)
print(Ps_mean)
print(Ps_std)

"""

# Media global = media de las medias
F2_mean = np.mean(F2s_mean)
Ec_mean = np.mean(Ecs_mean)
P_mean  = np.mean(Ps_mean)

# Error estándar de la media global --> ¿Deberia tener en cuenta las std de cada bloque simulado?
F2_err = np.std(F2s_mean, ddof=1) / np.sqrt(len(F2s_mean))
Ec_err = np.std(Ecs_mean, ddof=1) / np.sqrt(len(Ecs_mean))
P_err  = np.std(Ps_mean, ddof=1) / np.sqrt(len(Ps_mean))

print(f"<F²> = {F2_mean:.6f} ± {F2_err:.6f}")
print(f"<Ec> = {Ec_mean:.6f} ± {Ec_err:.10f}")
print(f"<P>  = {P_mean:.6f} ± {P_err:.10f}")

<F²> = 324.144055 ± 55.845157
<Ec> = 1020.245813 ± 0.0000000000
<P>  = 0.226950 ± 0.0000000000
